In [1]:
%reload_ext autoreload
%autoreload 2

import pytorch_lightning as pl
from dvclive.lightning import DVCLiveLogger
from pytorch_lightning.loggers import TensorBoardLogger

from src.data import OpenNeuroDataModule
from src.models import MultiHeadClassifier

In [33]:
datamodule = OpenNeuroDataModule(batch_size=2)
import torch
# this is a test
model = MultiHeadClassifier(n_subjects=datamodule.n_subjects,
                            n_spatial_features=12,
                            n_temporal_features=12,
                            example_input_array=datamodule.get_example_batch())

# x = datamodule.get_example_batch()[0]
x = torch.rand(64, 24, 10)
x_recon, h, y_cls, y_subj = model(x)

# trainer = pl.Trainer(
#     max_epochs=1, accelerator='auto', log_every_n_steps=1,
#     logger=[DVCLiveLogger(save_dvc_exp=True, report='md'),
#             TensorBoardLogger('.')]
#     )
# trainer.fit(model, datamodule=datamodule)

torch.Size([64, 24, 10])


(torch.Size([64, 24, 10]),
 torch.Size([64, 12]),
 torch.Size([64, 2]),
 torch.Size([64, 10]))

In [113]:
batch_size = 7
n_timepoints = 10
n_channels = 8
spatial_features = 4
temporal_features = 3

import torch
from torch.nn import LSTM, Conv1d, ConvTranspose1d

x_flat = torch.rand(batch_size, n_timepoints, n_channels).permute(0, 2, 1)

space_enc = Conv1d(n_channels, spatial_features, kernel_size=1)
time_enc = LSTM(1, temporal_features, batch_first=True)  
time_dec = LSTM(1, n_timepoints, batch_first=True)  
space_dec  = ConvTranspose1d(spatial_features, n_channels, kernel_size=1)

y_space_enc = space_enc(x_flat).permute(0, 2, 1)

h = []

for ch in range(spatial_features):
    x_ch = y_space_enc[:, :, ch].unsqueeze(-1)
    y_t_enc, (h_t_enc, c_t_enc) = time_enc(x_ch)
    h.append(h_t_enc[0,...])
    print(y_t_enc.shape, h_t_enc.shape, c_t_enc.shape)
    x_t_dec = torch.zeros(batch_size, n_timepoints, 1)
    y_t_dec, (h_t_dec, c_t_dec) = time_dec(x_t_dec, (h_t_enc, c_t_enc))
    x_ch = space_dec(y_t_dec)

h = torch.stack(h, dim=1)


torch.Size([7, 10, 3]) torch.Size([1, 7, 3]) torch.Size([1, 7, 3])


RuntimeError: Expected hidden[0] size (1, 7, 10), got [1, 7, 3]